## Proceso ETL

In [2]:
import pandas as pd
import numpy as np


In [3]:
data = pd.read_csv(r"C:\Users\Admin\Documents\Cars Sales\Archivo csv\Auto Sales data.csv", 
    parse_dates=['ORDERDATE'], 
    dayfirst=True)

In [4]:
#Creacion de la nueva columna DAYS_SINCE_LASTORDER basado en la diferencia de tiempo
#Primero elimino la columna existente

data = data.drop('DAYS_SINCE_LASTORDER', axis=1)

#Basado en la diferencia de tiempo creo la nueva columna DAYS_SINCE_LASTORDER

from datetime import datetime
#Especifico la fecha
temp_date = datetime(2020,6,1)
data['DAYS_SINCE_LASTORDER'] = (temp_date - data['ORDERDATE']).dt.days

In [5]:
data.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,CITY,POSTALCODE,COUNTRY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE,DAYS_SINCE_LASTORDER
0,10107,30,95.70,2,2871.00,2018-02-24,Shipped,Motorcycles,95,S10_1678,Land of Toys Inc.,2125557818,897 Long Airport Avenue,NYC,10022,USA,Yu,Kwai,Small,828
1,10121,34,81.35,5,2765.90,2018-05-07,Shipped,Motorcycles,95,S10_1678,Reims Collectables,26.47.1555,59 rue de l'Abbaye,Reims,51100,France,Henriot,Paul,Small,756
2,10134,41,94.74,2,3884.34,2018-07-01,Shipped,Motorcycles,95,S10_1678,Lyon Souveniers,+33 1 46 62 7555,27 rue du Colonel Pierre Avia,Paris,75508,France,Da Cunha,Daniel,Medium,701
3,10145,45,83.26,6,3746.70,2018-08-25,Shipped,Motorcycles,95,S10_1678,Toys4GrownUps.com,6265557265,78934 Hillside Dr.,Pasadena,90003,USA,Young,Julie,Medium,646
4,10168,36,96.66,1,3479.76,2018-10-28,Shipped,Motorcycles,95,S10_1678,Technics Stores Inc.,6505556809,9408 Furth Circle,Burlingame,94217,USA,Hirano,Juri,Medium,582


In [6]:
# Crear la tabla 'customers' sin duplicados y agregar 'customerid'
customers = data[['CUSTOMERNAME', 'PHONE', 'ADDRESSLINE1', 'CITY', 'POSTALCODE', 'COUNTRY', 
                'CONTACTLASTNAME', 'CONTACTFIRSTNAME']].drop_duplicates().reset_index(drop=True)
customers['customerid'] = range(1, len(customers) + 1)

# Crear la tabla 'orders' sin duplicados en 'ORDERNUMBER'
orders = data[['ORDERNUMBER', 'ORDERDATE', 'STATUS', 'CUSTOMERNAME']].drop_duplicates().reset_index(drop=True)

# Agregar 'customerid' a 'orders' usando 'CUSTOMERNAME' como clave de relación
orders = orders.merge(customers[['CUSTOMERNAME', 'customerid']], on='CUSTOMERNAME', how='left')

# Verificar si hay valores faltantes en 'customerid' en la tabla 'orders'
missing_customerid = orders['customerid'].isnull().sum()
orders.head(), customers.head(), missing_customerid


(   ORDERNUMBER  ORDERDATE   STATUS          CUSTOMERNAME  customerid
 0        10107 2018-02-24  Shipped     Land of Toys Inc.           1
 1        10121 2018-05-07  Shipped    Reims Collectables           2
 2        10134 2018-07-01  Shipped       Lyon Souveniers           3
 3        10145 2018-08-25  Shipped     Toys4GrownUps.com           4
 4        10168 2018-10-28  Shipped  Technics Stores Inc.           5,
            CUSTOMERNAME             PHONE                   ADDRESSLINE1  \
 0     Land of Toys Inc.        2125557818        897 Long Airport Avenue   
 1    Reims Collectables        26.47.1555             59 rue de l'Abbaye   
 2       Lyon Souveniers  +33 1 46 62 7555  27 rue du Colonel Pierre Avia   
 3     Toys4GrownUps.com        6265557265             78934 Hillside Dr.   
 4  Technics Stores Inc.        6505556809              9408 Furth Circle   
 
          CITY POSTALCODE COUNTRY CONTACTLASTNAME CONTACTFIRSTNAME  customerid  
 0         NYC      10022     USA 

In [14]:
# Agregar la columna 'SALES' a 'ordersdetails' usando 'ORDERNUMBER' y 'PRODUCTCODE' como claves de unión
ordersdetails = ordersdetails.merge(data[['ORDERNUMBER', 'PRODUCTCODE', 'SALES']], on=['ORDERNUMBER', 'PRODUCTCODE'], how='left')

# Verificar las primeras filas para confirmar que 'SALES' se agregó correctamente
print(ordersdetails.head())


   ORDERNUMBER PRODUCTCODE  QUANTITYORDERED  PRICEEACH  ORDERLINENUMBER  \
0        10107    S10_1678               30      95.70                2   
1        10107    S10_1678               30      95.70                2   
2        10121    S10_1678               34      81.35                5   
3        10121    S10_1678               34      81.35                5   
4        10121    S10_1678               34      81.35                5   

   orderdetailid  dealsizeid  productid   SALES  
0              1           1          1  2871.0  
1              1           2          1  2871.0  
2              2           1          1  2765.9  
3              2           3          1  2765.9  
4              2           2          1  2765.9  


In [10]:
# 1. Crear la tabla 'ordersdetails' con detalles únicos para cada pedido
ordersdetails = data[['ORDERNUMBER', 'PRODUCTCODE', 'QUANTITYORDERED', 'PRICEEACH', 'ORDERLINENUMBER']].drop_duplicates().reset_index(drop=True)
ordersdetails['orderdetailid'] = range(1, len(ordersdetails) + 1)

# 2. Crear la tabla 'products' con información única de cada producto
products = data[['PRODUCTCODE', 'PRODUCTLINE', 'MSRP']].drop_duplicates().reset_index(drop=True)
products['productid'] = range(1, len(products) + 1)

# 3. Crear la tabla 'dealsize' con valores únicos de 'DEALSIZE'
dealsize = data[['DEALSIZE']].drop_duplicates().reset_index(drop=True)
dealsize['dealsizeid'] = range(1, len(dealsize) + 1)

# 4. Relacionar 'dealsizeid' en 'ordersdetails' mediante un join
ordersdetails = ordersdetails.merge(data[['ORDERNUMBER', 'DEALSIZE']].drop_duplicates(), on='ORDERNUMBER', how='left')
ordersdetails = ordersdetails.merge(dealsize, on='DEALSIZE', how='left')

# Eliminar la columna redundante 'DEALSIZE' en 'ordersdetails'
ordersdetails = ordersdetails.drop(columns='DEALSIZE')

# Agregar 'productid' a 'ordersdetails' mediante una combinación con 'products' usando 'PRODUCTCODE'
ordersdetails = ordersdetails.merge(products[['PRODUCTCODE', 'productid']], on='PRODUCTCODE', how='left')

# Verificar las primeras filas para confirmar que 'productid' se agregó correctamente
print(ordersdetails.head())

# Mostrar las primeras filas de cada tabla
ordersdetails.head(), products.head(), dealsize.head()


   ORDERNUMBER PRODUCTCODE  QUANTITYORDERED  PRICEEACH  ORDERLINENUMBER  \
0        10107    S10_1678               30      95.70                2   
1        10107    S10_1678               30      95.70                2   
2        10121    S10_1678               34      81.35                5   
3        10121    S10_1678               34      81.35                5   
4        10121    S10_1678               34      81.35                5   

   orderdetailid  dealsizeid  productid  
0              1           1          1  
1              1           2          1  
2              2           1          1  
3              2           3          1  
4              2           2          1  


(   ORDERNUMBER PRODUCTCODE  QUANTITYORDERED  PRICEEACH  ORDERLINENUMBER  \
 0        10107    S10_1678               30      95.70                2   
 1        10107    S10_1678               30      95.70                2   
 2        10121    S10_1678               34      81.35                5   
 3        10121    S10_1678               34      81.35                5   
 4        10121    S10_1678               34      81.35                5   
 
    orderdetailid  dealsizeid  productid  
 0              1           1          1  
 1              1           2          1  
 2              2           1          1  
 3              2           3          1  
 4              2           2          1  ,
   PRODUCTCODE   PRODUCTLINE  MSRP  productid
 0    S10_1678   Motorcycles    95          1
 1    S10_1949  Classic Cars   214          2
 2    S10_2016   Motorcycles   118          3
 3    S10_4698   Motorcycles   193          4
 4    S10_4757  Classic Cars   136          5,
   DEA

In [17]:
# Primero, eliminamos cualquier duplicado en `ordersdetails` para `ORDERNUMBER` y `PRODUCTCODE`
ordersdetails = ordersdetails.drop_duplicates(subset=['ORDERNUMBER', 'PRODUCTCODE', 'QUANTITYORDERED', 'PRICEEACH', 'ORDERLINENUMBER'])

# Luego, agregamos de nuevo `dealsizeid`, asegurándonos de que esté basado en la combinación única
ordersdetails = ordersdetails.merge(dealsize, on='dealsizeid', how='left')

# Verificamos si los duplicados han sido eliminados
print(ordersdetails.head())


   ORDERNUMBER PRODUCTCODE  QUANTITYORDERED  PRICEEACH  ORDERLINENUMBER  \
0        10107    S10_1678               30      95.70                2   
1        10121    S10_1678               34      81.35                5   
2        10134    S10_1678               41      94.74                2   
3        10145    S10_1678               45      83.26                6   
4        10168    S10_1678               36      96.66                1   

   orderdetailid  dealsizeid  productid    SALES DEALSIZE  
0              1           1          1  2871.00    Small  
1              2           1          1  2765.90    Small  
2              3           2          1  3884.34   Medium  
3              4           2          1  3746.70   Medium  
4              5           2          1  3479.76   Medium  


In [19]:
ordersdetails = ordersdetails.drop(columns='DEALSIZE')


In [9]:
orders.head()

,ORDERNUMBER,ORDERDATE,STATUS,CUSTOMERNAME,customerid
0,10107,2018-02-24,Shipped,Land of Toys Inc.,1
1,10121,2018-05-07,Shipped,Reims Collectables,2
2,10134,2018-07-01,Shipped,Lyon Souveniers,3
3,10145,2018-08-25,Shipped,Toys4GrownUps.com,4
4,10168,2018-10-28,Shipped,Technics Stores Inc.,5


In [8]:
customers.head()

,CUSTOMERNAME,PHONE,ADDRESSLINE1,CITY,POSTALCODE,COUNTRY,CONTACTLASTNAME,CONTACTFIRSTNAME,customerid
0,Land of Toys Inc.,2125557818,897 Long Airport Avenue,NYC,10022,USA,Yu,Kwai,1
1,Reims Collectables,26.47.1555,59 rue de l'Abbaye,Reims,51100,France,Henriot,Paul,2
2,Lyon Souveniers,+33 1 46 62 7555,27 rue du Colonel Pierre Avia,Paris,75508,France,Da Cunha,Daniel,3
3,Toys4GrownUps.com,6265557265,78934 Hillside Dr.,Pasadena,90003,USA,Young,Julie,4
4,Technics Stores Inc.,6505556809,9408 Furth Circle,Burlingame,94217,USA,Hirano,Juri,5


In [11]:
products.head()

,PRODUCTCODE,PRODUCTLINE,MSRP,productid
0,S10_1678,Motorcycles,95,1
1,S10_1949,Classic Cars,214,2
2,S10_2016,Motorcycles,118,3
3,S10_4698,Motorcycles,193,4
4,S10_4757,Classic Cars,136,5


In [20]:
ordersdetails.head()

,ORDERNUMBER,PRODUCTCODE,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,orderdetailid,dealsizeid,productid,SALES
0,10107,S10_1678,30,95.70,2,1,1,1,2871.00
1,10121,S10_1678,34,81.35,5,2,1,1,2765.90
2,10134,S10_1678,41,94.74,2,3,2,1,3884.34
3,10145,S10_1678,45,83.26,6,4,2,1,3746.70
4,10168,S10_1678,36,96.66,1,5,2,1,3479.76


In [13]:
dealsize.head()

,DEALSIZE,dealsizeid
0,Small,1
1,Medium,2
2,Large,3


# Separacion de archivos

## Tabla pedidos (Orders)

## Tabla detalles orden (Orders_details)

## Tabla productos (Products)

## Tabla Clientes (Clients)

## Tabla Tamaño del trato (deal_size)

In [21]:
#Guardamos los dataframe en su respectivo arhivo csv para la importar a SQL 

orders.to_csv(r"C:\Users\Admin\Documents\Cars Sales\Archivo csv\Orders.csv", index=False)

ordersdetails.to_csv(r"C:\Users\Admin\Documents\Cars Sales\Archivo csv\OrdersDetails.csv", index=False)

products.to_csv(r"C:\Users\Admin\Documents\Cars Sales\Archivo csv\Products.csv", index=False)

customers.to_csv(r"C:\Users\Admin\Documents\Cars Sales\Archivo csv\Customers.csv", index=False)

dealsize.to_csv(r"C:\Users\Admin\Documents\Cars Sales\Archivo csv\DealSize.csv", index=False)